In [1]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")
import pandas as pd
import numpy as np
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import geopandas as gpd
import time
import requests
from cartoframes.viz import Map, Layer, popup_element
from pymongo import GEOSPHERE
db = client.get_database('Ironhack')

In [2]:
df_top = pd.read_csv('data/top_cities_final.csv')

df_top.head()

,Unnamed: 0,city,company,latitude,longitude,geometry
0,0,San Francisco,Digg,37.764726,-122.394523,"{'type': 'Point', 'coordinates': [37.764726, -..."
1,8,San Francisco,The Climate Corporation,37.782809,-122.394354,"{'type': 'Point', 'coordinates': [37.782809, -..."
2,10,New York,Gilt Groupe,40.747270,-73.980064,"{'type': 'Point', 'coordinates': [40.74727, -7..."
3,17,London,cScape,51.523676,-0.107843,"{'type': 'Point', 'coordinates': [51.523676, -..."
4,19,London,ProofHQ,51.532048,-0.011154,"{'type': 'Point', 'coordinates': [51.5320478, ..."


### Create a GEOSPHERE index in MongoDB

In [27]:
team_preferences = db.get_collection('team_preferences')

db.team_preferences.create_index([("geometry", GEOSPHERE)])

OperationFailure: Index build failed: 5627edaf-7fd3-41e6-bc30-0014fdae422f: Collection Ironhack.team_preferences ( c3880ef9-19fc-4ae2-9511-5d2407b43bbe ) :: caused by :: Can't extract geo keys: { _id: ObjectId('63ea4d07517b79e58f50c659'), name: "Pet Food Express", latitude: 37.78757, longitude: -122.446339, category: "basketball-court", geometry: { type: "Point", coordinates: [ 37.78757, -122.446339 ] } }  longitude/latitude is out of bounds, lng: 37.7876 lat: -122.446, full error: {'ok': 0.0, 'errmsg': 'Index build failed: 5627edaf-7fd3-41e6-bc30-0014fdae422f: Collection Ironhack.team_preferences ( c3880ef9-19fc-4ae2-9511-5d2407b43bbe ) :: caused by :: Can\'t extract geo keys: { _id: ObjectId(\'63ea4d07517b79e58f50c659\'), name: "Pet Food Express", latitude: 37.78757, longitude: -122.446339, category: "basketball-court", geometry: { type: "Point", coordinates: [ 37.78757, -122.446339 ] } }  longitude/latitude is out of bounds, lng: 37.7876 lat: -122.446', 'code': 16755, 'codeName': 'L

In [16]:
team_preferences.find_one(
        {"geometry":
            {"$geoIntersects": 
                {"$geometry": 
                     {'type': 'Point', 'coordinates':[40.740583,-74.003878 ] }}}},
    
projection = {"name": 1, "_id":0})




{'name': 'BeastyFeast'}

In [52]:
team_preferences_2 = db.get_collection("team_preferences_2")

In [134]:
def count_places (lon, lat):

    try:
        return len(list(team_preferences_2.find(
            {"geometry":
                {"$near": 
                    {"$geometry": 
                        {'type': 'Point', 'coordinates':[lon,lat] }, "$maxDistance": 200}}},
        
        projection = {"name": 1, "_id":0})))

    except:
        return 0

In [135]:
query

[]

In [136]:
test = df_top['longitude'][0], df_top['latitude'][0]
test

(-122.394523, 37.764726)

In [137]:
print(test)

(-122.394523, 37.764726)


In [138]:
count_places(df_top['longitude'][0], df_top['latitude'][0])

0

In [114]:
type(df_top['latitude'][0])

numpy.float64

In [112]:
df_top.sample()

,Unnamed: 0,city,company,latitude,longitude,geometry,count
33,679,San Francisco,Five Prime Therapeutics,37.767629,-122.393799,"{'type': 'Point', 'coordinates': [37.7676285, ...",0


In [139]:
df_top['count_2'] = df_top.apply(lambda row: count_places(row['longitude'], row['latitude']), axis=1)
df_top['count_2'].unique()

array([0, 2, 1, 3, 4], dtype=int64)

In [148]:
df_top.groupby("city")['count_2'].sum()

city
London            8
New York         12
San Francisco    17
Name: count_2, dtype: int64

In [151]:
df_top

,Unnamed: 0,city,company,latitude,longitude,geometry,count_2
0,0,San Francisco,Digg,37.764726,-122.394523,"{'type': 'Point', 'coordinates': [37.764726, -...",0
1,8,San Francisco,The Climate Corporation,37.782809,-122.394354,"{'type': 'Point', 'coordinates': [37.782809, -...",0
2,10,New York,Gilt Groupe,40.747270,-73.980064,"{'type': 'Point', 'coordinates': [40.74727, -7...",2
3,17,London,cScape,51.523676,-0.107843,"{'type': 'Point', 'coordinates': [51.523676, -...",1
4,19,London,ProofHQ,51.532048,-0.011154,"{'type': 'Point', 'coordinates': [51.5320478, ...",0
5,20,San Francisco,99designs,37.795531,-122.400598,"{'type': 'Point', 'coordinates': [37.7955307, ...",0
6,30,London,Sososher,51.500152,-0.126236,"{'type': 'Point', 'coordinates': [51.500152, -...",0
7,39,New York,PixelCrayons,28.682042,77.087502,"{'type': 'Point', 'coordinates': [28.682042, 7...",0
8,44,San Francisco,Cutcaster,37.798510,-122.457454,"{'type': 'Point', 'coordinates': [37.79851, -1...",0
9,46,New York,New York Designs,40.743264,-73.923427,"{'type': 'Point', 'coordinates': [40.743264, -...",0


In [145]:
df_top.sort_values("count_2", ascending=False)

,Unnamed: 0,city,company,latitude,longitude,geometry,count,count_2
20,276,New York,Fabian Rossano Studio,40.740371,-74.004777,"{'type': 'Point', 'coordinates': [40.740371, -...",0,4
19,267,London,Box UK,51.510774,-0.139245,"{'type': 'Point', 'coordinates': [51.5107737, ...",0,3
17,202,San Francisco,Ecolect,37.776769,-122.410111,"{'type': 'Point', 'coordinates': [37.776769, -...",0,3
15,170,San Francisco,Ecolect,37.776769,-122.410111,"{'type': 'Point', 'coordinates': [37.776769, -...",0,3
34,697,San Francisco,Sunrun,37.787131,-122.404107,"{'type': 'Point', 'coordinates': [37.7871306, ...",0,2
16,190,San Francisco,Milobox,37.792979,-122.421242,"{'type': 'Point', 'coordinates': [37.792979, -...",0,2
29,530,London,Boring,51.518372,-0.074068,"{'type': 'Point', 'coordinates': [51.5183725, ...",0,2
11,134,San Francisco,Minted,37.797435,-122.403175,"{'type': 'Point', 'coordinates': [37.797435, -...",0,2
37,728,San Francisco,ACT Biotech,37.787131,-122.404107,"{'type': 'Point', 'coordinates': [37.7871306, ...",0,2
18,222,San Francisco,Milobox,37.792979,-122.421242,"{'type': 'Point', 'coordinates': [37.792979, -...",0,2


In [92]:
list(team_preferences_2.find({"geometry":
{"$near": 
{"$geometry": 
    {'type': 'Point', 'coordinates':[-74.003878, 40.740583] }, 
        "$maxDistance": 200}}}, projection = {"name": 1, "_id":0}))

[{'name': 'BeastyFeast'}, {'name': 'Starbucks'}, {'name': 'Starbucks'}]

In [56]:
len([{'name': 'BeastyFeast'}, {'name': 'Starbucks'}, {'name': 'Starbucks'}])

3

In [ ]:
new_dict_venues = {"starbucks": 0,
}

In [18]:
coordinates = df_top['geometry']

In [20]:
def places_counts(coordinates,df):
    """
    Creates a dataframe with the result of the geoqueries
    Args:
        df (df): the dataframe where the columns will be added
        coordinates (list): the coordinates to be iterated over
    Returns:
        The new dataframe
    """
    party_counts = []
    starbucks_counts = []
    school_counts = []
    vegan_counts = []
    basket_counts = []
    for c in coordinates:
        query = {"geometry": {"$near": {"$geometry": c, "$maxDistance": 2000}}}
        list_ = list(team_preferences.find(query))
        party = 0
        starbucks = 0
        school = 0
        vegan = 0
        basket = 0
        for l in list_:
            if l['place'] == 'party':
                party += 1
            elif l['place'] == 'starbucks':
                starbucks += 1
            elif l['place'] == 'school':
                school += 1
            elif l['place'] == 'vegan':
                vegan += 1
            elif l['place'] == 'basket':
                basket += 1
        party_counts.append(party)
        starbucks_counts.append(starbucks)
        school_counts.append(school)
        vegan_counts.append(vegan)
        basket_counts.append(basket)

    df['party'] = party_counts
    df['starbucks'] = starbucks_counts
    df['school'] = school_counts
    df['vegan'] = vegan_counts
    df['basket'] = basket_counts

    return df
places_counts(coordinates,df_top)

OperationFailure: $geometry is required for geo near query, full error: {'ok': 0.0, 'errmsg': '$geometry is required for geo near query', 'code': 2, 'codeName': 'BadValue'}

### Add key places to each location in the dataframe


In [ ]:
def count_places(coordinates, df):
    """
    Creates a dataframe with the result of the geoqueries
    Args:
        df (df): the dataframe where the columns will be added
        coordinates (list): the coordinates to be iterated over
    Returns:
        The new dataframe
    """
    place_counts = {
        'party': [],
        'starbucks': [],
        'school': [],
        'vegan': [],
        'basket': []
    }
    
    for c in coordinates:
        query = {"location": {"$near": {"$geometry": c, "$maxDistance": 2000}}}
        locations = list(cp.find(query))
        
        for place_type in place_counts.keys():
            place_counts[place_type].append(sum(1 for loc in locations if loc['place'] == place_type))
    
    df = df.assign(**place_counts)
    return df

In [ ]:
count_places(coordinates, df)

OperationFailure: error processing query: ns=Ironhack.company_collectionTree: GEONEAR  field=location maxdist=2000 isNearSphere=0
Sort: {}
Proj: {}
 planner returned error :: caused by :: unable to find index for $geoNear query, full error: {'ok': 0.0, 'errmsg': 'error processing query: ns=Ironhack.company_collectionTree: GEONEAR  field=location maxdist=2000 isNearSphere=0\nSort: {}\nProj: {}\n planner returned error :: caused by :: unable to find index for $geoNear query', 'code': 291, 'codeName': 'NoQueryExecutionPlans'}

In [ ]:
df = count_places(coordinates,df)

OperationFailure: error processing query: ns=Ironhack.company_collectionTree: GEONEAR  field=location maxdist=2000 isNearSphere=0
Sort: {}
Proj: {}
 planner returned error :: caused by :: unable to find index for $geoNear query, full error: {'ok': 0.0, 'errmsg': 'error processing query: ns=Ironhack.company_collectionTree: GEONEAR  field=location maxdist=2000 isNearSphere=0\nSort: {}\nProj: {}\n planner returned error :: caused by :: unable to find index for $geoNear query', 'code': 291, 'codeName': 'NoQueryExecutionPlans'}

In [ ]:
places_counts(coordinates,df)

OperationFailure: error processing query: ns=Ironhack.company_collectionTree: GEONEAR  field=location maxdist=2000 isNearSphere=0
Sort: {}
Proj: {}
 planner returned error :: caused by :: unable to find index for $geoNear query, full error: {'ok': 0.0, 'errmsg': 'error processing query: ns=Ironhack.company_collectionTree: GEONEAR  field=location maxdist=2000 isNearSphere=0\nSort: {}\nProj: {}\n planner returned error :: caused by :: unable to find index for $geoNear query', 'code': 291, 'codeName': 'NoQueryExecutionPlans'}

In [ ]:

# 5. run your queries




In [34]:
len(team_preferences.find_one())

6

In [38]:
all_db = list(team_preferences.find({}))
all_db

[{'_id': ObjectId('63ea4d07517b79e58f50c52d'),
  'name': 'BeastyFeast',
  'latitude': 40.740583,
  'longitude': -74.003878,
  'category': 'basketball-court',
  'geometry': {'type': 'Point', 'coordinates': [40.740583, -74.003878]}},
 {'_id': ObjectId('63ea4d07517b79e58f50c52e'),
  'name': 'The Salty Paw',
  'latitude': 40.707577,
  'longitude': -74.001743,
  'category': 'basketball-court',
  'geometry': {'type': 'Point', 'coordinates': [40.707577, -74.001743]}},
 {'_id': ObjectId('63ea4d07517b79e58f50c52f'),
  'name': 'Biscuits & Bath',
  'latitude': 40.736371,
  'longitude': -73.996113,
  'category': 'basketball-court',
  'geometry': {'type': 'Point', 'coordinates': [40.736371, -73.996113]}},
 {'_id': ObjectId('63ea4d07517b79e58f50c530'),
  'name': "Eva's Play Pups",
  'latitude': 40.722154,
  'longitude': -73.959235,
  'category': 'basketball-court',
  'geometry': {'type': 'Point', 'coordinates': [40.722154, -73.959235]}},
 {'_id': ObjectId('63ea4d07517b79e58f50c531'),
  'name': 'Dog 

In [46]:
all_db[0]

{'_id': ObjectId('63ea4d07517b79e58f50c52d'),
 'name': 'BeastyFeast',
 'latitude': 40.740583,
 'longitude': -74.003878,
 'category': 'basketball-court',
 'geometry': {'type': 'Point', 'coordinates': [40.740583, -74.003878]}}

In [45]:
lon = all_db[0]['geometry']['coordinates'][1]
lat = all_db[0]['geometry']['coordinates'][0]
lat, lon

(40.740583, -74.003878)

In [50]:
new_list = []

for i in all_db:
    new_dict_ = {
        'name': i['name'],
        'lat': i['latitude'],
        'lon': i['longitude'],
        'category': i['category'],
        "geometry": {"type": "Point", "coordinates": [i['longitude'], i['latitude']]}

    }
    new_list.append(new_dict_)
new_list

[{'name': 'BeastyFeast',
  'lat': 40.740583,
  'lon': -74.003878,
  'category': 'basketball-court',
  'geometry': {'type': 'Point', 'coordinates': [-74.003878, 40.740583]}},
 {'name': 'The Salty Paw',
  'lat': 40.707577,
  'lon': -74.001743,
  'category': 'basketball-court',
  'geometry': {'type': 'Point', 'coordinates': [-74.001743, 40.707577]}},
 {'name': 'Biscuits & Bath',
  'lat': 40.736371,
  'lon': -73.996113,
  'category': 'basketball-court',
  'geometry': {'type': 'Point', 'coordinates': [-73.996113, 40.736371]}},
 {'name': "Eva's Play Pups",
  'lat': 40.722154,
  'lon': -73.959235,
  'category': 'basketball-court',
  'geometry': {'type': 'Point', 'coordinates': [-73.959235, 40.722154]}},
 {'name': 'Dog Room Club',
  'lat': 40.785382,
  'lon': -73.951538,
  'category': 'basketball-court',
  'geometry': {'type': 'Point', 'coordinates': [-73.951538, 40.785382]}},
 {'name': "Walter's Pet Styles",
  'lat': 40.745053,
  'lon': -73.978287,
  'category': 'basketball-court',
  'geometr

In [51]:
for i in new_list:
    db['team_preferences_2'].insert_one(i)
